# Install kaggle-environments

In [ ]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
#!curl -X PURGE https://pypi.org/simple/kaggle-environments

# ConnectX environment was defined in v0.1.6
!pip install 'kaggle-environments>=0.1.6'

# Create ConnectX Environment

In [ ]:
from kaggle_environments import evaluate, make, utils
from tqdm.notebook import tqdm
import numpy as np
import random
import collections
import json
import os
env = make("connectx", debug=True)
env.render()

# **Create Q-table**
load if applicable

In [ ]:
# file = "../input/q-dict-2000/q_dict (4).json"
# with open(file, "r") as f:
#     q_dict = json.load(f) 
#     #print(q_dict)
q_dict = {} 
file = "q_dict.json" 
with open(file, "w") as f: 
    json.dump(q_dict,f)


# **Getwinmove Helper Function**

In [ ]:
print(env.configuration)
def getwinmove(turn, board, con):
    cols = con.columns
    towin = con.inarow
    rows = con.rows
    colstreaks = [0]*cols
    collastgap = [-1]*cols
    for row in range(rows):
        streakpregap = 0
        streak = 0
        lastgap = -1
        for pos in range(cols*row,cols*row+cols):
            #print (pos)
            if board[pos] == turn:
                streak+=1
                colstreaks[pos%cols]+=1
            elif board[pos] == 0:
                lastgap = pos
                collastgap[pos%cols] = pos
                streakpregap = streak
                streak = 0
                colstreaks[pos%cols] = 0
            else:
                streak = 0
                streakpregap = 0
                lastgap = -1
                colstreaks[pos%cols] = 0
                collastgap[pos%cols] = -1
            if lastgap != -1:
                if streak + streakpregap >= towin-1:
                    #print(pos%cols,"row",row,board[pos])
                    if len(board)<= lastgap+cols or board[lastgap+cols]!=0:
                        return lastgap%cols
            if collastgap[pos%cols] !=-1:
                if colstreaks[pos%cols]>=towin-1:
                    #print(pos%cols,"col",board[collastgap[pos%cols]+cols])
                    if len(board)< collastgap[pos%cols]+cols or board[collastgap[pos%cols]+cols]!=0:
                        return pos%cols 
    return -1 
#print(getwinmove(1,[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],env.configuration))
                 
        
            
    

In [ ]:

games = 10000
alpha = .1
alpha_decrease = .9999
epsilon = 0.1
epsilon_decrease = .9999
gamma = 0.5
k = 1
def state_to_str(state):
    #print(state,state.mark,state["board"])
    return str(state["board"])
def getbest(lis,board):
    #f(u, n) = u + k/n
    best = -10000
    ops = []
    for i in range(0,env.configuration.columns*2,2):
        if board[i//2] == 0:
            u = lis[i]
            n = lis[i+1]
            val = u+k/n
            if val > best:
                best = val
                ops = [i]
            elif val == best:
                ops.append(i)
    if ops:
        return random.choice(ops)
    return 0
file = "q_dict.json"
with open(file) as f:
    q_dict = json.load(f)
#print (q_dict)
#for d in tqdm(range(1)):
if True:
    tot = 0
    #q_dict = dict()
    for i in tqdm(range(games)):
        trainer =  env.train([None, "negamax"])
        first = True
        if random.random() < .5:
            trainer = env.train(["negamax", None])
            frist = False
        state = trainer.reset()
        gameOver = False
        while not gameOver:
            boardstr = state_to_str(state)
            if boardstr not in q_dict:
                q_dict[boardstr]=[0,1]*env.configuration.columns
            winfirst = getwinmove(1,state["board"],env.configuration)
            winsecond = getwinmove(2,state["board"],env.configuration)
            if winfirst + winsecond > -2:
                if first:
                    if winfirst !=-1:
                        act= winfirst
                    else:
                        act = winsecond
                else:
                    if winsecond != -1:
                        act= winsecond
                    else:
                        act =  winfirst
                act*=2
            elif random.random() < epsilon:
                act = random.choice([c for c in range(env.configuration.columns) if state["board"][c] == 0])
                act*=2
            else:
                ops = []
                best = -10000
                 
                act = getbest(q_dict[boardstr],state["board"])

            next_state, reward, gameOver, other = trainer.step(act//2)
            #print(reward)
            
            if gameOver:
                if reward !=-1:
                    tot+=reward
                    #print(tot)
                reward*=25
                reward+=10

            else:
                reward = -.1
            state = next_state
            nextboardstr = state_to_str(next_state)
            if nextboardstr not in q_dict:
                q_dict[nextboardstr]=[0,1]*env.configuration.columns
            #Q(s,a) ← Q(s,a) +alpha[R(s,a,s') +  gamma*maxa'f(Q(s',a'),N(s',a')) -Q(s,a)]
            old_q = q_dict[boardstr][act]
            next_q = q_dict[nextboardstr][getbest(q_dict[nextboardstr],next_state["board"])]
            new_q = old_q + alpha*(reward+gamma*next_q - old_q)
            q_dict[boardstr][act+1]+=1
            q_dict[boardstr][act]=new_q
            alpha*=alpha_decrease
            epsilon*=epsilon_decrease



    

In [ ]:
file = "q_dict.json"
print()
with open(file, "w") as f:
    json.dump(q_dict,f)

In [ ]:
new_q_dict = dict()
for k in q_dict:
    #print(q_dict[k])
    best =  -10000
    ops = []
    ind = 0
    for i in range(0,env.configuration.columns*2,2):
        val = q_dict[k][i]
        if val > best:
                best = val
                ops = [i]
        elif val == best:
                ops.append(i)
    if ops:
        ind = random.choice(ops)
    new_q_dict[k]=ind//2
q_dict_str = str(new_q_dict)

# Create an Agent

To create the submission, an agent function should be fully encapsulated (no external dependencies).  

When your agent is being evaluated against others, it will not have access to the Kaggle docker image.  Only the following can be imported: Python Standard Library Modules, gym, numpy, scipy, pytorch (1.3.1, cpu only), and more may be added later.



In [ ]:

agent="""

import random
new_q_dict ="""+q_dict_str+"""

def my_agent(observation, configuration):   
    def getwinmove(turn, board, con):
        cols = con.columns
        towin = con.inarow
        rows = con.rows
        colstreaks = [0]*cols
        collastgap = [-1]*cols
        for row in range(rows):
            streakpregap = 0
            streak = 0
            lastgap = -1
            for pos in range(cols*row,cols*row+cols):
                #print (pos)
                if board[pos] == turn:
                    streak+=1
                    colstreaks[pos%cols]+=1
                elif board[pos] == 0:
                    lastgap = pos
                    collastgap[pos%cols] = pos
                    streakpregap = streak
                    streak = 0
                    colstreaks[pos%cols] = 0
                else:
                    streak = 0
                    streakpregap = 0
                    lastgap = -1
                    colstreaks[pos%cols] = 0
                    collastgap[pos%cols] = -1
                if lastgap != -1:
                    if streak + streakpregap >= towin-1:
                        print(pos%cols,"row",row,board[pos])
                        if len(board)<= lastgap+cols or board[lastgap+cols]!=0:
                            return lastgap%cols
                if collastgap[pos%cols] !=-1:
                    if colstreaks[pos%cols]>=towin-1:
                        print(pos%cols,"col")
                        if len(board)< collastgap[pos%cols]+cols or board[collastgap[pos%cols]+cols]!=0:
                            return pos%cols
        return -1
    board = observation.board
    boardkey = str(board)
    #print(observation)
    turnnum = observation.mark
    otherturnnum = (not (observation.mark -1)) +1
    h = getwinmove(turnnum, observation.board, configuration)
    print(h)
    if h != -1:
        return h
    h = getwinmove(otherturnnum, observation.board, configuration)
    print(h)
    if h != -1:
        return h
    if boardkey not in new_q_dict:
        return random.choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    else:
        action = new_q_dict[boardkey]
        #print(action)
    if observation.board[action] != 0:
        return random.choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    return action
"""


# Write Agent to File

In [ ]:

with open('submission.py', 'w') as f:
    f.write(agent)

In [ ]:
from submission import my_agent



# Test Agent

In [ ]:
# Play as first position against random agent.
trainer = env.train([None, "random"])

observation = trainer.reset()
print(observation)
while not env.done:
    my_action = my_agent(observation, env.configuration)
    print("My Action", my_action)
    #print(trainer)
    observation, reward, done, info = trainer.step(my_action)
    print(my_agent(observation, env.configuration))
    #print(trainer.step(my_action))
    env.render(mode="ipython", width=100, height=90, header=False, controls=False)
env.render()

# Evaluate Agent

In [ ]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=10)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [my_agent, "negamax"], num_episodes=10)))

# Validate Submission

In [ ]:
# Note: Stdout replacement is a temporary workaround.
import sys
out = sys.stdout
submission = utils.read_file("/kaggle/working/submission7.py")
agent = my_agent
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")